In [ ]:
# Pytorch Example:
!pip install cython
# Install pycocotools, the version by default in Colab
# has a bug fixed in https://github.com/cocodataset/cocoapi/pull/354
!pip install -U 'git+https://github.com/cocodataset/cocoapi.git#subdirectory=PythonAPI'

In [ ]:
#from PIL import Image
import pandas as pd
import numpy as np
import os
import math
#import matplotlib.image as mpimg
#from matplotlib import pyplot as plt
import json
#import os
#import numpy as np
import torch
import torch.utils.data
import torch.nn as nn
#from PIL import Image
import random
#import torch



In [ ]:
import matplotlib.image as mpimg
from matplotlib import pyplot as plt
from PIL import Image

In [ ]:
#%%shell

# Download TorchVision repo to use some files from
# references/detection
#git clone https://github.com/pytorch/vision.git
#cd vision
#&& git checkout v0.3.0

#cp references/detection/utils.py ../
#cp references/detection/transforms.py ../
#cp references/detection/coco_eval.py ../
#cp references/detection/engine.py ../
#cp references/detection/coco_utils.py ../
#print(sys.path)
sys.path.append("drive/My Drive/cs231n_personal_project/vision/references/detection")
from engine import train_one_epoch,evaluate
#from vision.references.detection.engine import train_one_epoch,evaluate
import utils
import transforms as T


from torchvision.transforms import functional as F
from torchvision.ops import misc as misc_nn_ops
sys.path.insert(0, "drive/My Drive/cs231n_personal_project/vision/torchvision")

#import torchvision
#from torchvision.
from models.detection.faster_rcnn import FastRCNNPredictor
from models.detection.mask_rcnn import MaskRCNNPredictor
import models
# for auto-reloading external modules
# see http://stackoverflow.com/questions/1907993/autoreload-of-modules-in-ipython
%load_ext autoreload
%autoreload 2

In [ ]:
dataDir = "drive/My Drive/cs231n_personal_project"
print(os.listdir(dataDir))

In [ ]:
with open(os.path.join(dataDir, 'label_descriptions.json'), 'r') as file:
    label_description = json.load(file)
#label_description
n_classes = len(label_description['categories'])
n_attributes = len(label_description['attributes'])
print(F"Classes: {n_classes}, Attributes: {n_attributes}")

In [ ]:
categories_df = pd.DataFrame(label_description['categories'])
attributes_df = pd.DataFrame(label_description['attributes'])

In [ ]:
#fashion_json = {}
#fashion_json["train"] = fashion_dict
#import json, os
#with open(os.path.join("output_16May2020", 'dataset_train.json'), 'w') as fp:
#    json.dump(fashion_json, fp)
fashion_dict = []
with open(os.path.join(dataDir, 'mini_dataset_train.json'), 'r') as file:
    fashion_json = json.load(file)
fashion_dict = fashion_json["train"]
#print(len(train_data_copy['ImageId'].unique().tolist()))
print(fashion_dict[6]['attributes'])

In [ ]:
class FashionDataset(torch.utils.data.Dataset):
    def __init__(self, dataset, image_dir, transforms=None):
        self.imgs = dataset
        self.image_dir = image_dir
        self.transforms = transforms
        #self.height = height
        #self.width = width
        # load all image files, sorting them to
        # ensure that they are aligned
        #self.imgs = list(sorted(os.listdir(os.path.join(root, "train"))))
        #self.masks = list(sorted(os.listdir(os.path.join(root, "PedMasks"))))

    def rle_decode_string(self, string, h, w):
        mask = np.full(h*w, 0, dtype=np.uint8)
        annotation = [int(x) for x in string.split(' ')]
        for i, start_pixel in enumerate(annotation[::2]):
            mask[start_pixel: start_pixel+annotation[2*i+1]] = 1
        mask = mask.reshape((h, w), order='F')
        return mask

    def rle2bbox(self, rle, shape):
        '''
        Get a bbox from a mask which is required for Detectron 2 dataset
        rle: run-length encoded image mask, as string
        shape: (height, width) of image on which RLE was produced
        Returns (x0, y0, x1, y1) tuple describing the bounding box of the rle mask
        
        Note on image vs np.array dimensions:
        
            np.array implies the `[y, x]` indexing order in terms of image dimensions,
            so the variable on `shape[0]` is `y`, and the variable on the `shape[1]` is `x`,
            hence the result would be correct (x0,y0,x1,y1) in terms of image dimensions
            for RLE-encoded indices of np.array (which are produced by widely used kernels
            and are used in most kaggle competitions datasets)
        '''
        
        a = np.fromiter(rle.split(), dtype=np.uint)
        a = a.reshape((-1, 2))  # an array of (start, length) pairs
        a[:,0] -= 1  # `start` is 1-indexed
        
        y0 = a[:,0] % shape[0]
        y1 = y0 + a[:,1]
        if np.any(y1 > shape[0]):
            # got `y` overrun, meaning that there are a pixels in mask on 0 and shape[0] position
            y0 = 0
            y1 = shape[0]
        else:
            y0 = np.min(y0)
            y1 = np.max(y1)
        
        x0 = a[:,0] // shape[0]
        x1 = (a[:,0] + a[:,1]) // shape[0]
        x0 = np.min(x0)
        x1 = np.max(x1)
        
        if x1 > shape[1]:
            # just went out of the image dimensions
            raise ValueError("invalid RLE or image dimensions: x1=%d > shape[1]=%d" % (
                x1, shape[1]
            ))

        return x0, y0, x1, y1
    
    def get_attr(self, attr_str):
        mask = np.zeros(341)
        #print(attr_str)
        if attr_str == '' or (isinstance(attr_str, float) and  math.isnan(attr_str)):
          return mask
        else:
          attr_list = [int(x) for x in attr_str.split(",")]
          mask[attr_list] = 1
        return mask

    def __getitem__(self, idx):
        # load images ad masks
        image_entry = self.imgs[idx]
        #masks = mask == obj_ids[:, None, None]
        img = Image.open(os.path.join(self.image_dir, image_entry['file_name'] + ".jpg")).convert("RGB")
        #img = img.resize()
        #boxes = torch.as_tensor(image_entry['boxes'], dtype=torch.float32)
        # there is only one class
        labels = torch.as_tensor(image_entry['labels'], dtype=torch.int64)
        #print(image_entry['labels'])
        masks = []
        boxes = []
        attributes = []
        for entry in image_entry['masks']:
            masks.append(self.rle_decode_string(entry, image_entry['height'], image_entry['width']))
            boxes.append(list(self.rle2bbox(entry, (image_entry['height'], image_entry['width']))))
        #print(image_entry["image_id"])
        for attr in image_entry['attributes']:
            attributes.append(self.get_attr(attr))
        #print("Anmol1: ",idx)
        attributes = torch.as_tensor(attributes, dtype=torch.int64)
        boxes = torch.as_tensor(boxes, dtype=torch.float32)
        #print("Anmol2")
        masks = torch.as_tensor(masks, dtype=torch.uint8)
        #print(masks.shape)
        #print("Image: ", np.array(img).shape)
        image_id = torch.tensor([idx])
        area = (boxes[:, 3] - boxes[:, 1]) * (boxes[:, 2] - boxes[:, 0])
        # suppose all instances are not crowd
        iscrowd = torch.zeros((len(image_entry['labels'])), dtype=torch.int64)

        target = {}
        target["boxes"] = boxes
        target["labels"] = labels
        target["masks"] = masks
        target["image_id"] = image_id
        target["area"] = area
        target["iscrowd"] = iscrowd
        target["attributes"] = attributes

        if self.transforms is not None:
            #print(img.size)
            img, target = self.transforms(img, target)

        return img, target

    def __len__(self):
        return len(self.imgs)

In [ ]:
test1 = FashionDataset(fashion_dict, os.path.join(dataDir, 'mini_kaggle', 'train'))
print(test1[0][1]["attributes"][0])

In [ ]:
#from models.detection.faster_rcnn import FastRCNNPredictor
#from models.detection.mask_rcnn import MaskRCNNPredictor

def get_instance_segmentation_model(num_classes):
    # load an instance segmentation model pre-trained on COCO
    model = models.detection.maskrcnn_resnet50_fpn(pretrained_backbone=True)

    # get the number of input features for the classifier
    in_features = model.roi_heads.box_predictor.cls_score.in_features
    # replace the pre-trained head with a new one
    model.roi_heads.box_predictor = FastRCNNPredictor(in_features, num_classes)

    # now get the number of input features for the mask classifier
    in_features_mask = model.roi_heads.mask_predictor.conv5_mask.in_channels
    hidden_layer = 256
    # and replace the mask predictor with a new one
    model.roi_heads.mask_predictor = MaskRCNNPredictor(in_features_mask,
                                                       hidden_layer,
                                                       num_classes)

    return model

In [ ]:
model = get_instance_segmentation_model(47)
#for param in model.parameters():
#    param.requires_grad = False
print(model)

In [ ]:
#import sys
#sys.path.insert(0, 'vision/references/detection')

#import transforms as T


class Resize(object):
    def __init__(self, min_size, max_size):
        if not isinstance(min_size, (list, tuple)):
            min_size = (min_size,)
        self.min_size = min_size
        self.max_size = max_size

    def __call__(self, image, target=None):
        #print(image.size)
        im_shape = torch.tensor(image.shape[-2:])
        #print(im_shape)
        min_size = float(torch.min(im_shape))
        max_size = float(torch.max(im_shape))
        #print("Min:", min_size, "Max:", max_size)
        scale_factor = self.min_size[0] / min_size
        #print("Scale factor: ", scale_factor)
        if max_size * scale_factor > self.max_size:
            scale_factor = self.max_size / max_size
        image = torch.nn.functional.interpolate(image[None], scale_factor=scale_factor, mode='bilinear',align_corners=False)[0]
        new_shape = torch.tensor(image.shape[-2:])
        #print("New Shape: ", new_shape)
        if target is None:
            return image, target

        if "masks" in target:
            mask = target["masks"]
            mask = misc_nn_ops.interpolate(mask[None].float(), scale_factor=scale_factor)[0].byte()
            target["masks"] = mask
        if "boxes" in target:
            ratios = [ torch.tensor(s, dtype=torch.float32) / torch.tensor(s_orig, dtype=torch.float32) 
                      for s, s_orig in zip(new_shape, im_shape)]
            ratio_height, ratio_width = ratios
            #ratio_height, ratio_width = float(new_shape[0])/float(im_shape[0]), new_shape[1]/im_shape[1]
            #print("Ratio Height: ", ratio_height, "Ratio Width: ", ratio_width)
            xmin, ymin, xmax, ymax = target["boxes"].unbind(1)
            #print("Xmin Before", xmin, xmin.dtype)
            xmin = xmin * ratio_width
            xmax = xmax * ratio_width
            ymin = ymin * ratio_height
            ymax = ymax * ratio_height
            #print("Xmin After: ", xmin)
            target["boxes"] = torch.stack((xmin, ymin, xmax, ymax), dim=1)
        return image, target


In [ ]:
#from engine import train_one_epoch,evaluate
#import utils
#import transforms as T
def get_transform(train):
    transforms = []
    # converts the image, a PIL image, into a PyTorch Tensor
    #transforms.append(T.Resize((400,), 600))
    transforms.append(T.ToTensor())
    transforms.append(Resize((400,), 600))
    if train:
        # during training, randomly flip the training images
        # and ground-truth for data augmentation
        transforms.append(T.RandomHorizontalFlip(0.5))
    return T.Compose(transforms)

In [ ]:
# use our dataset and defined transformations
dataset = FashionDataset(fashion_dict[:950], os.path.join(dataDir, 'mini_kaggle', 'train'),get_transform(train=True))
dataset_test = FashionDataset(fashion_dict[950:960], os.path.join(dataDir, 'mini_kaggle', 'train'), get_transform(train=False))

# split the dataset in train and test set
torch.manual_seed(1)
indices = torch.randperm(len(dataset)).tolist()
indices_test = torch.randperm(len(dataset_test)).tolist()
#print(len(indices))
#print(indices[101:102])
dataset = torch.utils.data.Subset(dataset, indices)
dataset_test = torch.utils.data.Subset(dataset_test, indices_test)

# define training and validation data loaders
data_loader = torch.utils.data.DataLoader(
    dataset, batch_size=4, shuffle=True, num_workers=4,
    collate_fn=utils.collate_fn)

data_loader_test = torch.utils.data.DataLoader(
    dataset_test, batch_size=1, shuffle=False, num_workers=4,
    collate_fn=utils.collate_fn)

In [ ]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
print(device)
# our dataset has two classes only - background and person
num_classes = 47

# get the model using our helper function
#model = get_instance_segmentation_model(num_classes)
# move model to the right device
model.to(device)

# construct an optimizer
params = [p for p in model.parameters() if p.requires_grad]
optimizer = torch.optim.SGD(params, lr=0.005,
                            momentum=0.9, weight_decay=0.0005)

# and a learning rate scheduler which decreases the learning rate by
# 10x every 3 epochs
lr_scheduler = torch.optim.lr_scheduler.StepLR(optimizer,
                                               step_size=3,
                                               gamma=0.1)

In [ ]:
import math
import sys
import time
import torch

#import torchvision.models.detection.mask_rcnn

#from coco_utils import get_coco_api_from_dataset
#from coco_eval import CocoEvaluator
#import utils
@torch.no_grad()
def evaluate2(model, data_loader, device):
    print("Testing - ANMOL")
    n_threads = torch.get_num_threads()
    # FIXME remove this and make paste_masks_in_image run on the GPU
    torch.set_num_threads(1)
    cpu_device = torch.device("cpu")
    model.eval()
    metric_logger = utils.MetricLogger(delimiter="  ")
    header = 'Test:'

    #ANMOLcoco = get_coco_api_from_dataset(data_loader.dataset)
    #ANMOLiou_types = _get_iou_types(model)
    #ANMOLcoco_evaluator = CocoEvaluator(coco, iou_types)
    res_list= []
    idx = 0
    for images, targets in metric_logger.log_every(data_loader, 100, header):
        images = list(img.to(device) for img in images)
        targets = [{k: v.to(device) for k, v in t.items()} for t in targets]

        torch.cuda.synchronize()
        model_time = time.time()
        outputs = model(images)

        outputs = [{k: v.to(cpu_device) for k, v in t.items()} for t in outputs]
        model_time = time.time() - model_time

        res = {target["image_id"].item(): output for target, output in zip(targets, outputs)}
        #print(res)
        res_list.append(res)
        evaluator_time = time.time()
        #ANMOLcoco_evaluator.update(res)
        evaluator_time = time.time() - evaluator_time
        metric_logger.update(model_time=model_time, evaluator_time=evaluator_time)

    # gather the stats from all processes
    metric_logger.synchronize_between_processes()
    print("Averaged stats:", metric_logger)
    print(res_list)
    #ANMOLcoco_evaluator.synchronize_between_processes()

    # accumulate predictions from all images
    #ANMOLcoco_evaluator.accumulate()
    #ANMOLcoco_evaluator.summarize()
    torch.set_num_threads(n_threads)
    return
    #ANMOLreturn coco_evaluator

In [ ]:
num_epochs = 3
import math
for epoch in range(num_epochs):
  # train for one epoch, printing every 10 iterations
  train_one_epoch(model, optimizer, data_loader, device, epoch, print_freq=10)
  # update the learning rate
  lr_scheduler.step()
  evaluate2(model, data_loader_test, device=device)

In [ ]:
torch.save(model.state_dict(), "output/pytorch_method.ckpt")